## Send e-mail to users to inform about dynamic services that will be deprecated

This notebook sends e-mails to notify users of a specific deployment about upcoming dynamic service deprecation. It should be self-explaining, but in case you like more verbosity, here's more information on how it works.

It does not work for comp. services.

**Inputs**:
- Service and version (you'll select the service from a dropdown menu).
- Deprecation date
- Deprecation reason

**Output**: E-mails are sent to the users. 
The deprecation is marked in the oSparc database. 
Optionally, you can test/debug with fake e-mails/your own e-mail.

**TODOs**:
1. Check if deprecation date is available in the database
2. Fetch deprecation date from the database
3. Notify users if they have they can do a simple patch version update (easy from the GUI) or not (they have to re-create the node). For now both cases are mentioned in the e-mail and users have to figure that out.
4. Potentially, we can have a predefined set of deprecation reasons to choose from.


In [ ]:
# EDIT HERE
deprecation_reason = 'Insufficient IT-Security for aging software component'

### Install dependencies

In [ ]:
%%capture

# Weird call to pip as suggested by PCR via akevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/#How-to-use-Pip-from-the-Jupyter-Notebook
import sys
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install psycopg2-binary
!{sys.executable} -m pip install asyncio
!{sys.executable} -m pip install aiopg
!{sys.executable} -m pip install typer
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --user --py widgetsnbextension #https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show

### Gather information on service-to-be-deprecated, deprecation date and reason

#### Select service from the DB

In [ ]:
# Input access variables
import sqlalchemy as db
import os, sys, getpass


PG_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
PG_ENDPOINT=os.environ.get('POSTGRES_ENDPOINT')
PG_DB=os.environ.get('POSTGRES_DB')
PG_USER=os.environ.get('POSTGRES_USER')

In [ ]:
# Get list of services from the database
pg_engine_url = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        user=PG_USER,
        password=PG_PASSWORD,
        database=PG_DB,
        host=PG_ENDPOINT.split(":")[0],
        port=int(PG_ENDPOINT.split(":")[1]),
    )

engine = db.create_engine(pg_engine_url)
connection = engine.connect()
metadata = db.MetaData()
services_table = db.Table('services_meta_data', metadata, autoload=True, autoload_with=engine)
query = db.select([services_table.c.key, services_table.c.version, services_table.c.deprecated])
services_list_all = engine.execute(query).fetchall()

# Sort by version
services_list_all= sorted(services_list_all, key = lambda x: x[1])
# Only keep services that don't have a deprecation date, and are dynamic
services_list = [x[:2] for x in services_list_all if x[2] == None and "simcore/services/dynamic/" in x[0]]
#

In [ ]:
from io import StringIO, BytesIO
import IPython.display
from IPython.display import clear_output
from pathlib import Path
from datetime import date
import ipywidgets
import ipywidgets as widgets
from ipywidgets import Layout, Button, VBox, Label, Box, GridBox, GridspecLayout, TwoByTwoLayout
row_box_layout = Layout(
    display="flex", flex_flow="row", align_items="stretch", border="none", width="100%"
)

column_box_layout = Layout(
    display="flex",
    flex_flow="column",
    align_items="stretch",
    border="none",
    width="100%",
)

class Viewer_Controlls:
    def __init__(self) -> None:

        self.e1_dropd: widgets.Dropdown
        self.e2_dropd: widgets.Dropdown
        self.deprecation_date: widgets.DatePicker
        
        
        self.options1 = sorted(list(set([x[0].split("/")[-1] for x in services_list])))

    def update_field(self):
        new_electrode_code = [
            str(self.e1_dropd.value),
            str(self.e2_dropd.value),
            str(self.e3_dropd.value),
        ]
        print(new_electrode_code)

    def create_controlls(self):
        #########################################################
        # Field Controls

        label__ = widgets.Label(
            value=str("To Be Deprecated:"),
            layout=Layout(width="10%"),
        )

        self.e1_dropd = widgets.Dropdown(
            options=self.options1,
            value=None,
            description="Service",
            disabled=False,
            layout=Layout(width="20%"),
        )

        self.e2_dropd = widgets.Dropdown(
            options=["All non-recent versions"],
            value=None,
            description="Version",
            disabled=False,
            layout=Layout(width="20%"),
        )

        self.deprecation_date = widgets.DatePicker(
            description='Pick a Date',
            disabled=False,
            value = date.today()
        )
        
        def on_change(change):
            self.e2_dropd.options = ["All non-recent versions"] + sorted([x[1] for x in services_list if self.e1_dropd.value == x[0].split("/")[-1] ])
        self.e1_dropd.observe(on_change)
        self.e2_dropd.observe(on_change)




        #########################################################
        # Compile View

        options_items = [
            label__,
            self.e1_dropd,
            self.e2_dropd,
            self.deprecation_date,
        ]
        box_options = Box(children=options_items, layout=row_box_layout)

        controller_children = [
            box_options
        ]
        return Box(children=controller_children, layout=column_box_layout)
viewerControl = Viewer_Controlls()
test_gui = viewerControl.create_controlls()
test_gui

---
# IMPORTANT: RUN FROM HERE ONWARDS AGAIN IF DROPDOWNS CHANGE

In [ ]:
selectedService = test_gui.children[0].children[1].value
selectedServiceKey = test_gui.children[0].children[1].value
selectedVersion = test_gui.children[0].children[2].value
selectedDate = test_gui.children[0].children[3].value
listOfVersions = [(x[0],x[1]) for x in services_list if selectedService == x[0].split('/')[-1]]
listOfCurrentServiceVersions = sorted([x[1] for x in listOfVersions])
assert(True if selectedVersion == "All non-recent versions" else selectedVersion in listOfCurrentServiceVersions)
#
#
if selectedVersion == "All non-recent versions":
    listOfToBeDeprecatedServiceVersions = listOfCurrentServiceVersions[:-1]
else:
    listOfToBeDeprecatedServiceVersions = [selectedVersion]

### Get project IDs containing services to be deprecated

In [ ]:
# Re-used code from osparc-simcore/scripts/maintenance/migrate_project/check_consistency_data. Thanks for that!
# This quickly gets all nodes info
from typing import Any, Dict
import aiopg, typer

async def _get_projects_nodes(pool) -> Dict[str, Any]:
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            await cursor.execute(
                "SELECT uuid, workbench, prj_owner, projects.name, users.name, users.email, last_change_date"
                ' FROM "projects"'
                " INNER JOIN users"
                " ON projects.prj_owner = users.id"
                " WHERE users.role != 'GUEST'"
            )
            typer.secho(
                f"found {cursor.rowcount} project rows, now getting project with valid node ids..."
            )
            project_db_rows = await cursor.fetchall()
        project_nodes = {
            project_uuid: {
                "nodes": list(workbench.values()),
                "owner": prj_owner,
                "study_name": name,
                "name": user_name,
                "email": user_email,
                "last_change_date": last_change_date,
            }
            for project_uuid, workbench, prj_owner, name, user_name, user_email, last_change_date in project_db_rows
            if len(workbench) > 0
        }
        typer.echo(
            f"processed {cursor.rowcount} project rows, found {len(project_nodes)} valid projects."
        )
        return project_nodes

async with aiopg.create_pool(
            f"dbname=simcoredb user={PG_USER} password={PG_PASSWORD} host={PG_ENDPOINT.split(':')[0]}"
        ) as pool:
            project_nodes = await _get_projects_nodes(pool)

In [ ]:
# Gather all the info and transform it into a pandas Dataframe
import pandas as pd
#print(selectedService)
#print(listOfToBeDeprecatedServiceVersions)
#print("---")
studiesAffectedList = []
for uuid, content in project_nodes.items():
    for node in content["nodes"]:
        if node["key"].split('/')[-1] == selectedService:
            if node["version"] in listOfToBeDeprecatedServiceVersions:
                #print(node["key"])
                print(content)
                #print(str(node["version"]))
                studiesAffectedList.append(
                    {"username": content["name"],
                    "email": content["email"],
                    "study_name": content["study_name"],
                    "node_name_version": f"{node['label']}, service version: {node['version']}",
                    "study_uuid": f"{uuid}", 
                     "last_change_date": f"{content['last_change_date']}"
                    }
                )
dfStudiesAffected=pd.DataFrame(studiesAffectedList)
dfStudiesAffected.head()

In [ ]:
# Merge entries corresponding to same study
#
#print(dfStudiesAffected)
if len(dfStudiesAffected.index) > 0:
    dfStudiesAffected = dfStudiesAffected['node_name_version'].groupby([dfStudiesAffected.username, dfStudiesAffected.email, dfStudiesAffected.study_name, dfStudiesAffected.study_uuid, dfStudiesAffected.last_change_date]).apply(list).reset_index()
    #print(df)
    #df.head()
#print("////")
#print(dfStudiesAffected)

### Send e-mails

In [ ]:
SMTP_HOST = os.environ.get('SMTP_HOST')
SMTP_PORT = int(os.environ.get('SMTP_PORT'))
SMTP_USERNAME = os.environ.get('SMTP_USERNAME')

#######################
SMTP_PASSWORD = os.environ.get('SMTP_PASSWORD')

#### Optional: Test with fake e-mails

In [ ]:
## For debugging: create dataframe with fake users and e-mails. It assumes previous output has at least 3 entries
if False:
    pd.options.mode.chained_assignment = None # To avoid warning
    df = dfStudiesAffected.head(1)
    df.username= ["tester"]
    
    # To receive the e-mail, you can substitute one fake e-mail with an existing one here:
    df.email = [ "tester@itis.swiss"]
else:
    df = dfStudiesAffected

In [ ]:
print("You are about to send emails to: ")
print('\n'.join(df.email.drop_duplicates()))
print("\nPlease be sure before continuing...")

In [ ]:
# This will send e-mails to fake users
import smtplib, ssl
from datetime import datetime
context = ssl.create_default_context()

with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
    server.starttls(context=context)
    server.login(SMTP_USERNAME, SMTP_PASSWORD)
    
    # Construct string with list of studies and services
    services_info = ""
    #print(df)
    for name, val in df.groupby(["username"]):
        if len(dfStudiesAffected.index) > 0:
            affectedStudiesString = ""
            affectedServiceVersionsString =""
            for version in listOfToBeDeprecatedServiceVersions:
                affectedServiceVersionsString += "- " + selectedService + " : version " + version + "\n"
            for studyname, nodenameversion, uuid, last_change_date in zip(val['study_name'].values,val['node_name_version'].values,val['study_uuid'].values,val['last_change_date'].values):
                affectedStudiesString += f"- {studyname} (Last Change Date: " + last_change_date.split(".")[0] + ")\n"
    
                # Compose message, including list of studies and services    
                message = f"""Subject: oSPARC Services deprecation
        
\nDear {name},
we noticied that some of your studies contain services in an older version that will be deprecated soon.

It has become necessary to phase out the following service's versions:

""" + affectedServiceVersionsString + """

We will require you to update by """ + str(selectedDate.strftime("%b %d, %Y")) + """ at the latest.

Your following projects are affected by this deprecation:

""" + affectedStudiesString + f"""

In all cases, a newer version of the affected services is available for you to use in your study.
Depending on the current version of your service, you will have to:
  - Use the GUI to update the service (if the current version can be updated to a new patch version).
  - Recreate the node (if a patch is not available).


You can find more information on how to update services here : https://docs.osparc.io/#/docs/study_setup/add_service?id=update-a-service 
After the deprecation date, you'll not be able to use those services in your studies. You will still be able to download the artifacts from the deprecated services.
Apologies for any inconvenience.

Cheers,
your friendly o²S²PARC team


-- This is an automatically composed e-mail alert --
-- Answer to {SMTP_USERNAME} in case of question --

"""
        
        email = val['email'].values
        server.sendmail(SMTP_USERNAME, email, message.encode('utf-8'))

# Set deprecation date in database

In [ ]:
from sqlalchemy import and_
metadata = db.MetaData()
smd = db.Table('services_meta_data', metadata, autoload=True, autoload_with=engine)

##########
#Inserting new record into database
for version in listOfToBeDeprecatedServiceVersions:
    print("Deprecating:","simcore/services/dynamic/" + selectedService,version)
    query = smd.update().\
           values(deprecated=selectedDate).\
           where(and_(smd.c.key=="simcore/services/dynamic/" + selectedService,smd.c.version==version))
    ResultProxy = connection.execute(query)
    print("Changed",ResultProxy.rowcount,"DB rows. Done.")
#ResultSet = ResultProxy.fetchall()